In [1]:
from simtk.openmm.app import*
from openforcefield.utils import*

In [2]:
from simtk.openmm.app import PDBFile
from openforcefield.utils import get_data_file_path

In [3]:
ls

Acetylacetone_04_July.py       Acetylacetone Exp-Copy.ipynb  README.md
Acetylacetone Exp-Copy1.ipynb  Acetylacetone.pdb


In [4]:
import os

pdb_file_path = os.path.join('Acetylacetone.pdb')
pdbfile = PDBFile(pdb_file_path)

# I got the pdb file from
# https://www.ebi.ac.uk/pdbe-srv/pdbechem/chemicalCompound/show/P2D
# renamed it, then I manually moved it to the data directory of openforcefield.

print(pdbfile)

In [5]:
from openforcefield.topology import Molecule
acetylacetone = Molecule.from_smiles('O=C(C)CC(=O)C')
print(acetylacetone)

# I got the SMILES-code from:
# https://en.wikipedia.org/wiki/Acetylacetone

Molecule with name '' and SMILES '[H][C]([H])([H])[C](=[O])[C]([H])([H])[C](=[O])[C]([H])([H])[H]'


In [6]:
print (pdb_file_path)

Acetylacetone.pdb


In [7]:
from openforcefield.topology import *
from openforcefield.typing.engines.smirnoff import ForceField

omm_topology =  pdbfile.topology
off_topology = Topology.from_openmm(omm_topology, unique_molecules=[acetylacetone])

forcefield = ForceField('openff-1.0.0.offxml')

system = forcefield.create_openmm_system(off_topology)


In [27]:
from simtk import *

# Langevin Dynamics:
time_step = 2*unit.femtoseconds
temperature = 300*unit.kelvin
friction = 1/unit.picosecond # collision rate
integrator = openmm.LangevinIntegrator(temperature, friction, time_step)

# Lenghth of simulation:
num_steps = 1

trj_freq = 1 # number of steps per written trajectory frame
data_freq = 1 #number of steps per fritten simulation statistics

# Set up OpenMM simulation:
simulation = openmm.app.Simulation (omm_topology, system, integrator)

# initial positions:
positions = pdbfile.getPositions()
simulation.context.setPositions(positions)

# Randomize the velocities from a Boltzmann distribution at a given temperature.
simulation.context.setVelocitiesToTemperature(temperature)

# MINIMIZE ENERGY: I deleted this line to get the non-minimized values:
simulation.minimizeEnergy()
minimized_coords = simulation.context.getState(getPositions=True).getPositions()
minimized_forces = simulation.context.getState(getForces=True).getForces()

# Configure information in output files:
#pdb_reporter = openmm.app.PDBReporter('trajectory.pdb', trj_freq)
#state_data_reporter = openmm.app.StateDataReporter ('data.csv', data_freq, step=True, potentialEnergy=True, temperature=True, density=True)
#simulation.reporters.append (pdb_reporter)
#simulation.reporters.append (state_data_reporter)
DCDReporter = openmm.app.DCDReporter ('traj.dcd', reportInterval=1)
simulation.reporters.append (DCDReporter)

In [9]:
help (openmm.app.DCDReporter)

Help on class DCDReporter in module simtk.openmm.app.dcdreporter:

class DCDReporter(builtins.object)
 |  DCDReporter(file, reportInterval, append=False, enforcePeriodicBox=None)
 |  
 |  DCDReporter outputs a series of frames from a Simulation to a DCD file.
 |  
 |  To use it, create a DCDReporter, then add it to the Simulation's list of reporters.
 |  
 |  Methods defined here:
 |  
 |  __del__(self)
 |  
 |  __init__(self, file, reportInterval, append=False, enforcePeriodicBox=None)
 |      Create a DCDReporter.
 |      
 |      Parameters
 |      ----------
 |      file : string
 |          The file to write to
 |      reportInterval : int
 |          The interval (in time steps) at which to write frames
 |      append : bool=False
 |          If True, open an existing DCD file to append to.  If False, create a new file.
 |      enforcePeriodicBox: bool
 |          Specifies whether particle positions should be translated so the center of every molecule
 |          lies in the sam

In [28]:
import time

print("Starting simulation")
start = time.process_time()

# Run the simulation
simulation.step(num_steps)

end = time.process_time()
print("Elapsed time %.2f seconds" % (end-start))
print("Hurra!")

Starting simulation
Elapsed time 0.00 seconds
Hurra!


In [11]:
ff_applied_parameters = forcefield.label_molecules(off_topology)[0]
ff_values=[]
ff_valuefile = open ('ff_valuefile.txt', 'w+')

for atoms, bonds in ff_applied_parameters['Bonds'].items():
    ff_valuefile.write (F'{atoms},{bonds}')
    ff_valuefile.write ('\n')
    
ff_valuefile.close()

In [12]:
import numpy

ff_valuefile = open ('ff_valuefile.txt', 'r')

ff_valuetab = numpy.genfromtxt(fname=ff_valuefile, dtype='unicode', delimiter='Lenght')

ff_valuefile.close

man_atom_a = [0,1,1,2,2,2,3,3,3,4,4,6,6,6]
man_atom_b = [1,2,3,7,8,9,4,10,11,5,6,12,13,14]

In [13]:
import numpy
import os

mincoorfile = os.path.join('trajectory.pdb')

mincoor = numpy.genfromtxt(fname=mincoorfile, skip_header=2, skip_footer=2, dtype='unicode')

print (mincoor)

[['HETATM' '1' 'C1' 'P2D' 'A' '1' '2.270' '-0.180' '0.038' '1.00' '0.00'
  'C']
 ['HETATM' '2' 'C2' 'P2D' 'A' '1' '1.024' '-0.840' '-0.556' '1.00' '0.00'
  'C']
 ['HETATM' '3' 'C3' 'P2D' 'A' '1' '-0.022' '-1.458' '0.330' '1.00' '0.00'
  'C']
 ['HETATM' '4' 'C4' 'P2D' 'A' '1' '-1.029' '-0.391' '0.594' '1.00' '0.00'
  'C']
 ['HETATM' '5' 'C5' 'P2D' 'A' '1' '-2.247' '-0.216' '-0.280' '1.00'
  '0.00' 'C']
 ['HETATM' '6' 'O2' 'P2D' 'A' '1' '0.861' '-0.814' '-1.775' '1.00' '0.00'
  'O']
 ['HETATM' '7' 'O4' 'P2D' 'A' '1' '-0.828' '0.378' '1.529' '1.00' '0.00'
  'O']
 ['HETATM' '8' 'H1' 'P2D' 'A' '1' '2.787' '-0.894' '0.683' '1.00' '0.00'
  'H']
 ['HETATM' '9' 'H2' 'P2D' 'A' '1' '2.875' '0.194' '-0.792' '1.00' '0.00'
  'H']
 ['HETATM' '10' 'H3' 'P2D' 'A' '1' '1.972' '0.706' '0.605' '1.00' '0.00'
  'H']
 ['HETATM' '11' 'H4' 'P2D' 'A' '1' '-0.474' '-2.318' '-0.171' '1.00'
  '0.00' 'H']
 ['HETATM' '12' 'H5' 'P2D' 'A' '1' '0.434' '-1.829' '1.252' '1.00' '0.00'
  'H']
 ['HETATM' '13' 'H6' 'P2D' 'A'

In [14]:
xyz_data = mincoor[:,6:9]
xyz_data = xyz_data.astype(numpy.float)
print (xyz_data)

xa = xyz_data[man_atom_a,0]
xb = xyz_data[man_atom_b,0]
ya = xyz_data[man_atom_a,1]
yb = xyz_data[man_atom_b,1]
za = xyz_data[man_atom_a,2]
zb = xyz_data[man_atom_b,2]

[[ 2.27  -0.18   0.038]
 [ 1.024 -0.84  -0.556]
 [-0.022 -1.458  0.33 ]
 [-1.029 -0.391  0.594]
 [-2.247 -0.216 -0.28 ]
 [ 0.861 -0.814 -1.775]
 [-0.828  0.378  1.529]
 [ 2.787 -0.894  0.683]
 [ 2.875  0.194 -0.792]
 [ 1.972  0.706  0.605]
 [-0.474 -2.318 -0.171]
 [ 0.434 -1.829  1.252]
 [-2.906 -1.087 -0.227]
 [-2.875  0.602  0.08 ]
 [-1.949 -0.073 -1.321]]


In [15]:
bondlength_min=numpy.sqrt(((xa-xb)**2)+((ya-yb)**2)+((za-zb)**2))

print (bondlength_min)    

[1.53001699 1.50367417 2.39560222 2.88672583 3.51860725 2.9554284
 1.50931276 2.14629425 2.15433911 3.50032756 2.37463218 3.08995874
 2.51793288 3.09556812]


In [29]:
import mdtraj as md

t = md.load('traj.dcd', top='Acetylacetone.pdb')
print (t)

<mdtraj.Trajectory with 1 frames, 15 atoms, 1 residues, without unitcells>


In [40]:
from __future__ import print_function
print ('How many atoms %s' %t.n_atoms)
print (t.n_residues)

How many atoms 15
1


In [43]:
frame_idx = 0
atom_idx = 9
print ('Where is 9th Atom')
print ('x; %s\ty: %s\tz: %s' %tuple(t.xyz[frame_idx, atom_idx,:]))

Where is 9th Atom
x; 0.19394386	y: 0.06841549	z: 0.06106008
